In [ ]:
import os; os._exit(0) # 학습 후 저장이 끝나면 커널을 죽여서 VRAM 비우기

In [ ]:
# # train/test 비율 수동 조정
# import shutil
# import os

# src_folder = os.path.join(os.getcwd(), 'datasets/BALL/ball/test/good')
# dst_folder = os.path.join(os.getcwd(), 'datasets/BALL/ball/train/good')

# # 이동할 파일 번호 범위 설정
# for i in range(1001, 8431):
#     src_file = os.path.join(src_folder, f"{i}.png")
#     dst_file = os.path.join(dst_folder, f"{i}.png")
    
#     if os.path.exists(src_file):  # 파일이 존재하는 경우만 이동
#         shutil.move(src_file, dst_file)

In [ ]:
import warnings
from torchvision import transforms

warnings.filterwarnings("ignore")

# from custom.utils import # Enhancing, ROI, roi_upper, roi_left, roi_lower, roi_right
from custom.data import Ball, MVTec
from custom.models import Patchcore, EfficientAd, Detector
from custom.engine import Engine

method = 'EfficientAd' # Patchcore EfficientAd Dinomaly Detector
category = 'enhanced_ball' # bottle cable capsule carpet grid hazelnut leather metal_nut pill screw tile toothbrush transistor wood zipper

options = {}
if method != 'Patchcore':
    options['train_batch_size'] = 1
    if 'ball' in category:
        options['category'] = category

transform = transforms.Compose([
    # ROI(roi_upper),  
    # Enhancing(),
])
  
if transform.transforms:
    options['transform'] = transform

datamodule = Ball(**options) if category == 'ball' or category == 'enhanced_ball' else MVTec(**options)
model = Patchcore() if method == 'Patchcore' else EfficientAd() if method == 'EfficientAd' else Detector

engine = None
if method == 'patchcore':
    engine = Engine()
elif method == 'Detector':
    engine = Engine(is_detector=True)
elif method == 'EfficientAd':
    engine = Engine(max_epochs=10)


In [ ]:
engine.fit(datamodule=datamodule, model=model) # Train the model

In [ ]:
predictions = engine.predict(
    datamodule=datamodule,
    model=model,
    ckpt_path=f'./results/{method}/Ball/{category}/v0/weights/lightning/model.ckpt',
    return_predictions=True,
)

engine.report()

In [ ]:
import json
from sklearn.metrics import classification_report, roc_auc_score

if method != 'Detector':
    
    # 1. 필요한 키만 남긴 새로운 딕셔너리 생성
    filtered_predictions = {
        "image_path": [],
        "label": [],
        "pred_labels": []
    }

    for batch in predictions:
        # 각 배치에서 필요한 키만 추출하여 통합
        filtered_predictions["image_path"].extend(batch["image_path"])
        filtered_predictions["label"].extend(batch["label"].tolist())
        filtered_predictions["pred_labels"].extend(batch["pred_labels"].tolist())
        # filtered_predictions["pred_scores"].extend(batch["pred_scores"].tolist())

    # 2. 결과를 로컬에 저장
    output_path = f"./results/{method}/Ball/{category}/v0/predictions.json"
    with open(output_path, "w") as f:
        json.dump(filtered_predictions, f, indent=4)

    print(f"Filtered predictions saved to {output_path}")
    
    """옛날 결과를 다시 불러오고 싶으면 여기서 부터 실행"""
    # output_path = f"./results/{method}/Ball/{category}/v0/predictions.json"
    # with open(output_path, "r") as f:
    #     json.dump(filtered_predictions, f, indent=4)

    # 실제 라벨과 예측 라벨 추출
    true_labels = []
    predicted_labels = []
    predicted_scores = []

    for batch in predictions:
        true_labels.extend(batch['label'].cpu().numpy())  # 실제 라벨
        predicted_labels.extend(batch['pred_labels'].cpu().numpy())  # 예측 라벨
        predicted_scores.extend(batch['pred_scores'].cpu().numpy())  # 예측 점수

    # 성능 평가
    report = classification_report(true_labels, predicted_labels, target_names=['Normal', 'Anomalous'], digits=4)
    print("Classification Report:")
    print(report)

    # AUROC 계산
    auroc = roc_auc_score(true_labels, predicted_scores)
    print(f"AUROC: {auroc:.4f}")

Filtered predictions saved to ./results/EfficientAd/Ball/enhanced_ball/v0/predictions.json
Classification Report:
              precision    recall  f1-score   support

      Normal     0.9964    0.9584    0.9770      1754
   Anomalous     0.9599    0.9966    0.9779      1754

    accuracy                         0.9775      3508
   macro avg     0.9782    0.9775    0.9775      3508
weighted avg     0.9782    0.9775    0.9775      3508

AUROC: 0.9935
